# Part 1: Scraping [metrolyrics.com](http://www.metrolyrics.com/top-artists.html)

In [5]:
#importing all the needed libraries 
import requests
from lxml import html
from lxml.etree import tostring
import time
import requests
from pprint import pprint
from bs4 import BeautifulSoup

#textblob
from textblob import TextBlob, Word
import numpy as np #for numeric operations
import pandas as pd #for dealing with dataframes
import matplotlib.pyplot as plt #for visualization
import re
import time
from lxml.cssselect import CSSSelector
from lxml import html
import json
from selenium import webdriver
import time
import pandas as pd
from itertools import chain
from glob import glob

In [6]:
class scraper:
    def __init__(self,url):
        self.url=url
        self.response=requests.get(self.url)
        self.page=self.response.content
    def beaut(self):
        page=BeautifulSoup(self.page, "html.parser")
        return page
    def s_lxml(self):
        tree = html.document_fromstring(self.page)
        return tree
    def selen(self):
        browser = webdriver.Chrome()
        browser=browser.get(self.url)
        return browser      

In [7]:
def function_1(url):
    '''Function 1 scrapes all the popular song texts from a particular artist '''
    each=scraper(url).beaut()# making the page a BeautifulSoup from the above defined class
    links=[]
    divs=each.find("div",id="popular") # finding table of popular songs of a particular artist 
    urls=[i.get("href") for i in divs.select("td a")] #retrieving the urls of all popular songs of an artist
    artist=each.select_one("div h1").get_text().replace("Lyrics","").strip() # getting the name of an artist from the header
    for i in urls:
        links.append(scraper(i).beaut())
        time.sleep(3) #making song urls a bs4 object
         # getting texts of all popular songs of a particular artist
    text=[",".join([i.get_text() for i in j.find_all("p",class_="verse")]) for j in links]
    divs=each.find("div",id="popular") 
    unclean_rating=[i.get("style") for i in divs.find_all("span",style=re.compile(".*width.+"))] 
    ratings=[re.findall("[0-9.]+",i)[0] for i in unclean_rating] #getting song rating
    z=each.find_all("script")[5].get_text() 
    genre=re.findall('"tag",.+',z) #getting song genre/in a tag/
    df=pd.read_html(url)[0] #making a dataframe from all the obtained data
    df["Popularity"]=ratings
    df["genre"]=genre[0]
    df = df.rename(columns={'Unnamed: 0': 'Artist'})
    df["lyrics"]=text
    df["Artist"]=artist
    return df

In [8]:
def function_2(url="http://www.metrolyrics.com/artists-1.html"):
    '''function_2 gets top artists from the given url and returns back all their songs using function_1 '''
    art_page=scraper(url).beaut() #making the url a bs4 object
    divs=art_page.find_all("div",class_=re.compile("artist grid_2")) # finding all divs of artists matching "artist grid_2"
    links=[i.find("a").get("href") for i in divs] #getting links for all top artists separate pages
    frames=[]
    for i in links:
        frames.append(function_1(i))
        print(i)
        time.sleep(10)
    result=pd.concat(frames,ignore_index=True) #taking into account that different artists have diverse number of songs
    #using function_1 to get all song texts with ratings and genres for top artists of a noted url
    return result

In [9]:
def char_range(c1, c2):
    """Generates the characters from c1 to c2, inclusive.
     char_range function will be used to separately run the code using alphabetical order.""" 
    for c in range(ord(c1), ord(c2)+1):
        yield chr(c)

In [10]:
def function_3(start_char,end_char,first=False):
    '''function_3 gets all artists' urls and returns all their songs with respective names, ratings, song texts, 
      genres and popularity for noted range of alphabet letters '''
    frames=[]
    base=["http://www.metrolyrics.com/artists-{}.html".format(i) for i in char_range(start_char,end_char)]
    if first==True:
        base=["http://www.metrolyrics.com/artists-{}.html".format(i) for i in chain(char_range(start_char,end_char),["1"])]
    for i in base:
        frames.append(function_2(i))
        time.sleep(20)
        print(i)
    result=pd.concat(frames,ignore_index=True)
    return result

In [ ]:
df1=function_3("a","e",True) #this df1 returns all top artists'respective song data names of which are in from a to e, inclusive.
#also this df1 includes names starting with numbers, e.g. 50 cent, which is done by giving the first=True in function_3.

http://www.metrolyrics.com/atif-aslam-lyrics.html
http://www.metrolyrics.com/abba-lyrics.html
http://www.metrolyrics.com/arijit-singh-lyrics.html
http://www.metrolyrics.com/adele-lyrics.html
http://www.metrolyrics.com/ar-rahman-lyrics.html
http://www.metrolyrics.com/annie-lyrics.html
http://www.metrolyrics.com/artists-a.html
http://www.metrolyrics.com/brett-young-lyrics.html
http://www.metrolyrics.com/beatles-lyrics.html


In [52]:
df1.to_csv("df1.csv",index=False) #making artists' data a csv file.

In [187]:
"""df2=function_3("f","j")
df3=function_3("k","o")
df4=function_3("p","u")
df5=function_3("v","z")""" # we divided the code into 5 parts and run on 5 computers to get all top artists' data from a-z.
